In [160]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [161]:
def convert_columns_to_str(df,number_columns):
    """
    Функция для конвертации указанных столбцов в строковый тип и очистки от пробельных символов в начале и конце
    """
    
    for column in number_columns: # Перебираем список нужных колонок
        try:            
            df.iloc[:,column] = df.iloc[:,column].astype(str)
            # Очищаем колонку от пробельных символов с начала и конца
            df.iloc[:,column] = df.iloc[:,column].apply(lambda x: x.strip())
        except IndexError:
            print('Проверьте порядковые номера колонок которые вы хотите обработать.')
        

def convert_params_columns_to_int(lst):
    """
    Функция для конвератации значений колонок которые нужно обработать.
    Очищает от пустых строк, чтобы в итоге остался список из чисел в формате int
    """
    out_lst = [] # Создаем список в который будем добавлять только числа
    for value in lst: # Перебираем список
        try:
            # Обрабатываем случай с нулем, для того чтобы после приведения к питоновскому отсчету от нуля не получилась колонка с номером -1
            number = int(value)
            if number != 0:
                out_lst.append(value) # Если конвертирования прошло без ошибок то добавляем
            else:
                continue
        except: # Иначе пропускаем
            continue
    return out_lst
                


In [162]:
file_params = 'params.xlsx'
path_to_end_folder = 'data/'

In [163]:
first_df = pd.read_excel('data/Большой список.xlsx')
second_df = pd.read_excel('data/Малый список.xlsx')
# Считываем файл параметров без заголовка, чем проще тем лучше, пропущенные значения обозначим пустыми строками
params = pd.read_excel(file_params,header=None,keep_default_na=False)

In [164]:
params

,0,1
0,1,0
1,2,
2,3,
3,,
4,,
5,,ФИО
6,,
7,,
8,,
9,,


In [165]:
# Преврашаем каждую колонку в список
params_first_columns = params[0].tolist()
params_second_columns = params[1].tolist()



In [166]:
params_second_columns

[0, '', '', '', '', 'ФИО', '', '', '', '', '', '', '', '', '', '', 4]

In [167]:
# Конвертируем в инт заодно проверяя корректность введенных данных
int_params_first_columns = convert_params_columns_to_int(params_first_columns)
int_params_second_columns = convert_params_columns_to_int(params_second_columns)

    


In [168]:
int_params_first_columns

[1, 2, 3]

In [169]:
int_params_second_columns

[4]

In [170]:
# Отнимаем 1 от каждого значения чтобы привести к питоновским индексам
int_params_first_columns = list(map(lambda x:x-1,int_params_first_columns))
int_params_second_columns = list(map(lambda x:x-1,int_params_second_columns))

In [171]:
int_params_second_columns

[3]

In [172]:
# Конвертируем нужные нам колонки в str
convert_columns_to_str(first_df,int_params_first_columns)
convert_columns_to_str(second_df,int_params_second_columns)

In [173]:
# Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
first_df['ID'] = first_df.iloc[:,int_params_first_columns].sum(axis=1)
second_df['ID'] = second_df.iloc[:,int_params_second_columns].sum(axis=1)


In [174]:
# очищаем от пробелов между словами
first_df['ID'] =first_df['ID'].apply(lambda x:x.replace(' ',''))
second_df['ID'] =second_df['ID'].apply(lambda x:x.replace(' ',''))

In [175]:
first_df['ID']

0            АлехинДанилаПрокопьевич
1            АлехинДанилаПрокопьевич
2           ЭсауловаВалерияПрокловна
3               ПанинГавриилВласович
4       ГрибановКонстантинСтепанович
                    ...             
998       НиконоваАнтонинаМаксимовна
999        ФурмановАркадийЕлизарович
1000        КорниецЛюдмилаМаксимовна
1001       СеребровГавриилФеодосивич
1002            ЦойВалентинЗахарович
Name: ID, Length: 1003, dtype: object

In [176]:
second_df['ID']

0           АлехинДанилаПрокопьевич
1          ЭсауловаВалерияПрокловна
2              ПанинГавриилВласович
3      ГрибановКонстантинСтепанович
4          МандрыкоТрофимЛеонидович
                   ...             
436      НиконоваАнтонинаМаксимовна
437       ФурмановАркадийЕлизарович
438        КорниецЛюдмилаМаксимовна
439       СеребровГавриилФеодосивич
440       СеребровГавриилФеодосивич
Name: ID, Length: 441, dtype: object

In [177]:
# Обрабатываем дубликаты

duplicates_first_df = first_df[first_df.duplicated(subset=['ID'],keep=False)] # Сохраняем все значения у которых есть дубликаты в отдельный датафрейм


first_df.drop_duplicates(subset=['ID'],keep=False,inplace=True)# Удаляем дубликаты из датафрейма

duplicates_second_df = second_df[second_df.duplicated(subset=['ID'],keep=False)] # Сохраняем все значения у которых есть дубликаты в отдельный датафрейм
second_df.drop_duplicates(subset=['ID'],keep=False,inplace=True)# Удаляем дубликаты из датафрейма


In [178]:
duplicates_first_df

,Фамилия,Имя,Отчество,ФИО,Дата рождения,Номер телефона,Электронная почта,Адрес прописки,Страна,Регион,...,ИНН ФЗ/ИП,СНИЛС,ОМС,Специальность,Направление,Учебное заведение,Серия/Номер диплома,Регистрационный номер,Дата окончания обучения,ID
0,Алехин,Данила,Прокопьевич,Алехин Данила Прокопьевич,14.04.1973,NaN,NaN,"Россия, г. Иркутск, Кирова ул., д. 1 кв.212",Россия,Сахалинская область,...,158493217864,NaN,5950614844448865,Администратор базы данных,02.03.03 Математическое обеспечение и админист...,Санкт-Петербургский государственный университет,714531 1469061,11-701-701,2019,АлехинДанилаПрокопьевич
1,Алехин,Данила,Прокопьевич,Алехин Данила Прокопьевич,14.04.1973,NaN,NaN,"Россия, г. Иркутск, Кирова ул., д. 1 кв.212",Россия,Сахалинская область,...,158493217864,NaN,5950614844448865,Администратор базы данных,02.03.03 Математическое обеспечение и админист...,Санкт-Петербургский государственный университет,714531 1469061,11-701-701,2019,АлехинДанилаПрокопьевич
945,Янчина,Алиса,Емельяновна,Янчина Алиса Емельяновна,28.10.1961,NaN,NaN,"Россия, г. Магнитогорск, Лесной пер., д. 9 кв.49",Россия,Калининградская обл.,...,941642278308,NaN,2917853394988924,Тюремный надзиратель,40.03.01 Юриспруденция,Южный федеральный университет,955233 5566349,97-620-492,2003,ЯнчинаАлисаЕмельяновна
946,Янчина,Алиса,Емельяновна,Янчина Алиса Емельяновна,28.10.1961,NaN,NaN,"Россия, г. Магнитогорск, Лесной пер., д. 9 кв.49",Россия,Калининградская обл.,...,941642278308,NaN,2917853394988924,Тюремный надзиратель,40.03.01 Юриспруденция,Южный федеральный университет,955233 5566349,97-620-492,2003,ЯнчинаАлисаЕмельяновна
955,Цой,Валентин,Захарович,Цой Валентин Захарович,02.11.1981,NaN,NaN,"Россия, г. Новороссийск, Колхозная ул., д. 11 ...",Россия,Нижегородская область,...,496482379574,NaN,4293105134740472,Водитель,23.05.02 Транспортные средства специального на...,Российский университет дружбы народов,212552 9580652,69-087-839,2014,ЦойВалентинЗахарович
1002,Цой,Валентин,Захарович,Цой Валентин Захарович,02.11.1981,NaN,NaN,"Россия, г. Новороссийск, Колхозная ул., д. 11 ...",Россия,Нижегородская область,...,496482379574,NaN,4293105134740472,Водитель,23.05.02 Транспортные средства специального на...,Российский университет дружбы народов,212552 9580652,69-087-839,2014,ЦойВалентинЗахарович


In [179]:
# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Данные которых нет во 2 таблице'
wb.create_sheet(title='Данные которых нет во 1 таблице', index=1)
wb.create_sheet(title='Совпадающие данные', index=2)
# Создаем листы для дубликатов
wb.create_sheet(title='Дубликаты первая таблица', index=3)
wb.create_sheet(title='Дубликаты вторая таблица', index=4)

# Создаем датафрейм
itog_df = pd.merge(first_df, second_df, how='outer', left_on=['ID'], right_on=['ID'],
                   indicator=True)

# Записываем каждый датафрейм в соответсвующий лист
left_df = itog_df[itog_df['_merge'] == 'left_only']


left_df.drop(['_merge'], axis=1, inplace=True)
for r in dataframe_to_rows(left_df, index=False, header=True):
    wb['Данные которых нет во 2 таблице'].append(r)

right_df = itog_df[itog_df['_merge'] == 'right_only']
right_df.drop(['_merge'], axis=1, inplace=True)
for r in dataframe_to_rows(right_df, index=False, header=True):
    wb['Данные которых нет во 1 таблице'].append(r)

both_df = itog_df[itog_df['_merge'] == 'both']
both_df.drop(['_merge'], axis=1, inplace=True)
for r in dataframe_to_rows(both_df, index=False, header=True):
    wb['Совпадающие данные'].append(r)

# Записываем дубликаты в соответствующие листы
for r in dataframe_to_rows(duplicates_first_df, index=False, header=True):
    wb['Дубликаты первая таблица'].append(r)

for r in dataframe_to_rows(duplicates_second_df, index=False, header=True):
    wb['Дубликаты вторая таблица'].append(r)

In [180]:
# Сохраняем
t = time.localtime()
current_time = time.strftime('%H_%M_%S %d.%m.%Y', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder}/Совпадающие,уникальные данные  от {current_time}.xlsx')